# Image Segmentation of Cell Membranes

Given the image characteristics (bright cell membranes on dark background), we'll use watershed segmentation after finding markers using distance transform.

## Import required libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, filters, morphology, segmentation, feature, data
from scipy import ndimage

## Load and preprocess the image

In [2]:
img = data.cells3d()[0, 1]  # Using a sample image from skimage
# Normalize image
img_norm = (img - img.min()) / (img.max() - img.min())
plt.imsave('normalized.png', img_norm)

## Create binary mask using adaptive thresholding

In [3]:
thresh = filters.threshold_local(img_norm, block_size=35)
binary = img_norm > thresh
plt.imsave('binary.png', binary.astype(np.uint8) * 255)

## Find markers for watershed using distance transform

In [4]:
distance = ndimage.distance_transform_edt(~binary)
local_max = feature.peak_local_max(distance, min_distance=20, labels=~binary)
markers = np.zeros_like(distance, dtype=int)
markers[tuple(local_max.T)] = np.arange(1, len(local_max) + 1)
markers = ndimage.label(markers)[0]

## Apply watershed segmentation

In [5]:
segmented = segmentation.watershed(-distance, markers, mask=~binary)

# Save result
plt.figure(figsize=(10, 10))
plt.imshow(segmented, cmap='nipy_spectral')
plt.axis('off')
plt.savefig('segmentation_result.png', bbox_inches='tight', dpi=300)
plt.close()